In [1]:
import dataverk as dv
import pandas as pd
import ipywidgets as widgets
from os import environ
import plotly.graph_objs as go
import plotly.io as plio

/Users/B149899/anaconda3/envs/dv/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/B149899/anaconda3/envs/dv/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# Get data 

In [2]:
conn = dv.connectors.JSONStatConnector()
df = conn.get_pandas_df('https://data.ssb.no/api/v0/no/table/09665')

2019-06-14T15:51:55.671331: B149899 https://data.ssb.no/api/v0/no/table/09665


In [3]:
df.head(5)

,kjønn,type sykefravær,statistikkvariabel,kvartal,value
0,Begge kjønn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2000K2,6.9
1,Begge kjønn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2000K3,7.1
2,Begge kjønn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2000K4,7.1
3,Begge kjønn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2001K1,7.6
4,Begge kjønn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2001K2,7.0


# Transform data 

In [4]:
df_e = df[(df['kjønn']!='Begge kjønn') & (df['statistikkvariabel']=='Sykefraværsprosent, ikke-sesongjustert')]
df_m = df[(df['kjønn']=='Menn') & (df['type sykefravær']=='Egenmeldt og legemeldt') & (df['statistikkvariabel']=='Sykefraværsprosent, ikke-sesongjustert')]
df_k = df[(df['kjønn']=='Kvinner') & (df['type sykefravær']=='Egenmeldt og legemeldt') & (df['statistikkvariabel']=='Sykefraværsprosent, ikke-sesongjustert')]

In [5]:
df_m.head(5)

,kjønn,type sykefravær,statistikkvariabel,kvartal,value
1575,Menn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2000K2,5.7
1576,Menn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2000K3,6.0
1577,Menn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2000K4,5.8
1578,Menn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2001K1,6.2
1579,Menn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2001K2,5.7


# Create data views

In [6]:
data = [ go.Scatter(
            x=df_m['kvartal'], 
            y=df_m['value'],
            name='Menn'
            ),
        go.Scatter(
            x=df_k['kvartal'], 
            y=df_k['value'],
            name='Kvinner'
            )
       ]

layout = go.Layout(title='Sykefravær i norge')
fig_1 = go.Figure(data,layout)

go.FigureWidget(data,layout)

FigureWidget({
    'data': [{'name': 'Menn',
              'type': 'scatter',
              'uid': '7a969729-8…

In [7]:
data = [ go.Bar(
     x=df_m['kvartal'], 
     y=df_m['value'],
     name='Menn'
),
go.Bar(
    x=df_k['kvartal'], 
    y=df_k['value'],
    name='Kvinner'
)
]

layout = go.Layout(
    barmode='stack'
)

layout = go.Layout(title='Sykefravær')
fig_2 = go.Figure(data,layout)

go.FigureWidget(data,layout)

FigureWidget({
    'data': [{'name': 'Menn',
              'type': 'bar',
              'uid': '19c4f353-5879-…

# Create datapackage

In [30]:
readme = """
# SSB Sykefravær
Sykefravær for lønnstakere, etter kjønn og type sykefravær. Sesongjustert og ikke-sesongjustert (prosent) (avslutta serie) 2000K2 - 2018K4

kilde: ['SSB tabell 09665']('https://www.ssb.no/statbank/table/09665')
"""

metadata = {
'store': 'gs',
'repo': 'https://raw.githubusercontent.com/datasett/nav-examples/master',
'title': 'SSB Sykefravær',
'readme': readme,
'license':'MIT',
'accessRights':'Open',
'auth': 'NAV',
'description':'SSB Sykefravær',
'name':'SSB Sykefravær',
'source':'NAV',
'keywords':['Sykefravær, SSB'],
'provenance':'SSB',
'publisher': 'NAV', 
'user':'nav-examples',
'bucket': 'dataparcel',
'dataset':'ssb-sykefravaer',
'project': 'mesiqi'
}

In [31]:
dp = dv.Datapackage(metadata)

### Add views 

In [32]:
desc = '''
### Sykefravær.

Egenmeldt og legemeldt. Sykefraværsprosent, ikke-sesongjustert
'''

dp.add_view(
    spec_type='plotly',
    name='Sykefravær',
    title='Sykefravær',
    description =desc,
    spec= plio.to_json(fig_1),
    resources=''
)

In [33]:
desc = '''
### Sykefravær.

Egenmeldt og legemeldt. Sykefraværsprosent, ikke-sesongjustert
'''

dp.add_view(
    spec_type='plotly',
    name='Sykefravær',
    title='Sykefravær',
    description =desc,
    spec= plio.to_json(fig_2),
    resources=''
)

### Add some resource files

In [34]:
dp.add_resource(df_e,
                'Egenmeldt og legemeldt',
                'Egenmeldt og legemeldt')

dp.add_resource(df_m,
                'Sykefravær menn. Egenmeldt og legemeldt. Sykefraværsprosent, ikke-sesongjustert',
                'Sykefravær menn. Egenmeldt og legemeldt. Sykefraværsprosent, ikke-sesongjustert')

dp.add_resource(df_k,
                'Sykefravær kvinner. Egenmeldt og legemeldt. Sykefraværsprosent, ikke-sesongjustert',
                'Sykefravær kvinner. Egenmeldt og legemeldt. Sykefraværsprosent, ikke-sesongjustert')

### Add a table view

In [35]:
df_e.head()

,kjønn,type sykefravær,statistikkvariabel,kvartal,value
1575,Menn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2000K2,5.7
1576,Menn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2000K3,6.0
1577,Menn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2000K4,5.8
1578,Menn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2001K1,6.2
1579,Menn,Egenmeldt og legemeldt,"Sykefraværsprosent, ikke-sesongjustert",2001K2,5.7


In [36]:
desc = '''
## Sykefravær Menn 

Menn. Egenmeldt og legemeldt (prosent)
'''

dp.add_view(
    spec_type='table',
    description = desc,
    name='Egenmeldt og legemeldt',
    title='Egenmeldt og legemeldt',
    resources = 'Sykefravær menn. Egenmeldt og legemeldt. Sykefraværsprosent, ikke-sesongjustert'
)

In [37]:
desc = '''
## Sykefravær Kvinner 

Menn. Egenmeldt og legemeldt (prosent)
'''

dp.add_view(
    spec_type='table',
    description = desc,
    name='Egenmeldt og legemeldt',
    title='Egenmeldt og legemeldt',
    resources = 'Sykefravær kvinner. Egenmeldt og legemeldt. Sykefraværsprosent, ikke-sesongjustert'
)

In [38]:
desc = '''
## Sykefravær

Egenmeldt og legemeldt (prosent)
'''

dp.add_view(
    spec_type='table',
    description = desc,
    name='Egenmeldt og legemeldt',
    title='Egenmeldt og legemeldt',
    resources = 'Egenmeldt og legemeldt'
)

# Save datafiles and publish datapackage in elasticsearch index

In [39]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/users/b149899/projects/binder/binder/keys/mesiqi.json'

In [40]:
#GOOGLE_APPLICATION_CREDENTIALS environment variable must be defined before initializing the client
client = dv.Client() 
client.publish(dp)

gs
2019-06-14T15:54:04.903542: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: json) written to https://storage.googleapis.com/dataparcel/ed7b5a1e8b39a82961d8bca0c2138f34/datapackage.json
2019-06-14T15:54:05.326144: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: csv) written to https://storage.googleapis.com/dataparcel/ed7b5a1e8b39a82961d8bca0c2138f34/resources/Egenmeldt%20og%20legemeldt.csv
2019-06-14T15:54:05.664056: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: csv) written to https://storage.googleapis.com/dataparcel/ed7b5a1e8b39a82961d8bca0c2138f34/resources/Sykefrav%C3%A6r%20menn.%20Egenmeldt%20og%20legemeldt.%20Sykefrav%C3%A6rsprosent%2C%20ikke-sesongjustert.csv
2019-06-14T15:54:06.028738: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: csv) written to https://storage.googleapis.com/dataparcel/ed7b5